In [2]:
import pandas as pd
import re
import xlutils
import time
import xlrd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import excelSave as save

In [4]:
df = pd.read_csv('filter708brands.csv')
df = pd.DataFrame(df)
df.head()

,Unnamed: 0,Unnamed: 0.1,brand_name,brand_name_split,brandCN,brandEN
0,0,0,canmake/井田,"['canmake', '井田']",井田,canmake
1,1,1,SUQQU,['SUQQU'],NaN,SUQQU
2,2,2,three,['three'],NaN,three
3,3,3,Paula‘s Choice/宝拉珍选,"['Paula‘s Choice', '宝拉珍选']",宝拉珍选,Paula‘s Choice
4,4,4,Laroche Posay/理肤泉,"['Laroche Posay', '理肤泉']",理肤泉,Laroche Posay
5,5,5,Elizabeth Arden/雅顿,"['Elizabeth Arden', '雅顿']",雅顿,Elizabeth Arden
6,6,6,FANCL,['FANCL'],芳珂,FANCL
7,7,7,rimmel,['rimmel'],NaN,rimmel
8,8,8,Milani,['Milani'],NaN,Milani
9,9,9,Curel/珂润,"['Curel', '珂润']",珂润,Curel


In [ ]:
brand_list = []
for split_lst in df['brand_name_split'].tolist():
    brand_list.append(split_lst[0])
brand_df = pd.DataFrame(data = brand_list)
brand_df.to_csv('brand_name.txt',header = None, index = None)

In [5]:
lst = list()

# roll up/down
def Transfer_Clicks(browser):
    time.sleep(5)
    try:
        browser.execute_script("window.scrollBy(0,document.body.scrollHeight)", "")
    except:
        pass
    return "Transfer successfully \n"

# load page
def isPresent():
    temp =1
    try: 
        driver.find_elements_by_css_selector('div.line-around.layout-box.mod-pagination > a:nth-child(2) > div > select > option')
    except:
        temp =0
    return temp

# insert data
def insert_data(elems,path,name,yuedu,taolun):
    for elem in elems:
        workbook = xlrd.open_workbook(path)  
        sheets = workbook.sheet_names()  
        worksheet = workbook.sheet_by_name(sheets[0])  
        rows_old = worksheet.nrows   
        rid = rows_old
       
        weibo_username = elem.find_elements_by_css_selector('h3.m-text-cut')[0].text
        weibo_userlevel = "普通用户"
        
        # user level
        try: 
            weibo_userlevel_color_class = elem.find_elements_by_css_selector("i.m-icon")[0].get_attribute("class").replace("m-icon ","")
            if weibo_userlevel_color_class == "m-icon-yellowv":
                weibo_userlevel = "黄v"
            if weibo_userlevel_color_class == "m-icon-bluev":
                weibo_userlevel = "蓝v"
            if weibo_userlevel_color_class == "m-icon-goldv-static":
                weibo_userlevel = "金v"
            if weibo_userlevel_color_class == "m-icon-club":
                weibo_userlevel = "微博达人"     
        except:
            weibo_userlevel = "普通用户"
        
        # post content
        weibo_content = elem.find_elements_by_css_selector('div.weibo-text')[0].text
        shares = elem.find_elements_by_css_selector('i.m-font.m-font-forward + h4')[0].text
        comments = elem.find_elements_by_css_selector('i.m-font.m-font-comment + h4')[0].text
        likes = elem.find_elements_by_css_selector('i.m-icon.m-icon-like + h4')[0].text

        # post time
        weibo_time = elem.find_elements_by_css_selector('span.time')[0].text
        '''
        print("用户名："+ weibo_username + "|"
              "微博等级："+ weibo_userlevel + "|"
              "微博内容："+ weibo_content + "|"
              "转发："+ shares + "|"
              "评论数："+ comments + "|"
              "点赞数："+ likes + "|"
              "发布时间："+ weibo_time + "|"
              "话题名称" + name + "|" 
              "话题讨论数" + yuedu + "|"
              "话题阅读数" + taolun)
        '''
        value1 = [[rid, weibo_username, weibo_userlevel,weibo_content, shares,comments,likes,weibo_time,keywords,name,taolun,yuedu],]
        print("当前插入第%d条数据" % rid)
        save.write_excel_xls_append_norepeat(path, value1)
        
# extract data
def get_current_weibo_data(elems,final_book_name,name,yuedu,taolun,maxWeibo):

        before = 0 
        after = 0
        n = 0 
        timeToSleep = 100
        while True:
            before = after
            Transfer_Clicks(driver)
            time.sleep(3)
            elems = driver.find_elements_by_css_selector('div.card.m-panel.card9')
            print("当前包含微博最大数量：%d,n当前的值为：%d, n值到5说明已无法解析出新的微博" % (len(elems),n))
            after = len(elems)
            if after > before:
                n = 0
            if after == before:        
                n = n + 1
            if n == 5:
                print("当前关键词最大微博数为：%d" % after)
                insert_data(elems,final_book_name,name,yuedu,taolun)
                break
            if len(elems)>maxWeibo:
                print("当前微博数以达到%d条"%maxWeibo)
                insert_data(elems,final_book_name,name,yuedu,taolun)
                break
            '''
            if after > timeToSleep:
                print("抓取到%d多条，插入当前新抓取数据并休眠5秒" % timeToSleep)
                timeToSleep = timeToSleep + 100
                insert_data(elems,book_name_xls,name,yuedu,taolun) 
                time.sleep(5) 
            '''


def spider(username,password,driver,book_name_xls,sheet_name_xls,keywords,maxWeibo):
    
    # create file
    if os.path.exists(book_name_xls):
        print("文件已存在")
    else:
        print("文件不存在，重新创建")
        value_title = [["rid", "用户名称", "微博等级", "微博内容", "微博转发量","微博评论量","微博点赞","发布时间","搜索关键词","话题名称","话题讨论数","话题阅读数"],]
        for keyword in keywords:
            final_book_name = keyword + book_name_xls
            save.write_excel_xls(final_book_name, sheet_name_xls, value_title)
    
    # open the specified web page
    driver.set_window_size(452, 790)
    #driver.get("https://passport.weibo.cn/signin/login?entry=mweibo&res=wel&wm=3349&r=https%3A%2F%2Fm.weibo.cn%2F")
    driver.get("https://passport.weibo.cn/signin/login")
    #driver.get("https://m.weibo.cn")

    print("开始自动登陆，若出现验证码手动验证")
    time.sleep(3)
    
    
    # log in
    elem = driver.find_element_by_xpath("//*[@id='loginName']");
    elem.send_keys(username)
    elem = driver.find_element_by_xpath("//*[@id='loginPassword']");
    elem.send_keys(password)
    elem = driver.find_element_by_xpath("//*[@id='loginAction']");
    elem.send_keys(Keys.ENTER)  
    print("暂停20秒，用于验证码验证")
    time.sleep(20)

    '''
    # 添加cookie
    cookie = []
    for ix in cookie:
        driver.add_cookie(ix)
    driver.get("https://m.weibo.cn")
    '''
    
    for keyword in keywords:
        final_book_name = keyword + book_name_xls
        
        # decide whether the web page has finished loading
        while 1:
            result = isPresent()
            print ('判断页面1成功 0失败  结果是=%d' % result )
            if result == 1:
                elems = driver.find_elements_by_css_selector('div.line-around.layout-box.mod-pagination > a:nth-child(2) > div > select > option')
                # return elems 
                break
            else:
                print ('页面还没加载出来呢')
                time.sleep(20)
        time.sleep(5)
        
        # search key words
        try:
            elem = driver.find_element_by_xpath("//*[@class='m-text-cut']").click();
            time.sleep(2)
            elem = driver.find_element_by_xpath("//*[@type='search']");
            elem.send_keys(keyword)
            elem.send_keys(Keys.ENTER)

            time.sleep(2)
            for i in range(100,500,100): # roll down until reach the button of super topic
                command = "window.scrollBy(0,"+str(i)+")"
                driver.execute_script(command, "")
                try:
                    driver.find_element_by_class_name('m-img-icon').click()
                    break
                except:
                    pass
            print("超话链接获取完毕，休眠2秒")
            time.sleep(2)

            yuedu_taolun = driver.find_element_by_xpath("//*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]").text
            yuedu = yuedu_taolun.split("　")[0]
            taolun = yuedu_taolun.split("　")[1]
            time.sleep(2)
            name = keyword
            shishi_element = driver.find_element_by_xpath("//*[@class='scroll-box nav_item']/ul/li/span[text()='帖子']")
            driver.execute_script('arguments[0].click()',shishi_element)
            get_current_weibo_data(elems,final_book_name,name,yuedu,taolun,maxWeibo) #爬取实时
            time.sleep(2)
            driver.get("https://m.weibo.cn")
            time.sleep(5)
            
        except Exception as e:
            print(keyword,e)
            lst.append(keyword)
            driver.get("https://m.weibo.cn")
            time.sleep(5)
            continue


    
if __name__ == '__main__':
    username = "bengyouwm022435@163.com" 
    password = "uz942976" 
    driver = webdriver.Firefox(executable_path = 'geckodriver')
    # with open('brand_name.txt', 'r',encoding='utf-8') as f:
    #    df = f.readlines()
    df1 = df['brandCN'].tolist()
    df1+=df['brandEN'].tolist()
    df1 = [x for x in df1 if pd.notnull(x)]
    df1 = [x.strip() for x in df1]
    keywords = df1
    book_name_xls =  ".xls" 
    sheet_name_xls = '微博数据' 
    maxWeibo = 10 
    spider(username, password, driver, book_name_xls, sheet_name_xls, keywords, maxWeibo)

文件不存在，重新创建
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！

xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数据成功！
xls格式表格写入数

井田 Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
宝拉珍选 Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
当前包含微博最大数量：20,n当前的值为：0, n值到5说明已无法解析出新的微博
当前微博数以达到10条
当前插入第1条数据
目前是第一行
xls格式表格【追加】写入数据成功！
当前插入第2条数据
xls格式表格【追加】写入数据成功！
当前插入第3条数据
xls格式表格【追加】写入数据成功！
当前插入第4条数据
xls格式表格【追加】写入数据成功！
当前插入第5条数据
xls格式表格【追加】写入数据成功！
当前插入第6条数据
xls格式表格【追加】写入数据成功！
当前插入第7条数据
xls格式表格【追加】写入数据成功！
当前插入第8条数据
xls格式表格【追加】写入数据成功！
当前插入第9条数据
xls格式表格【追加】写入数据成功！
当前插入第10条数据
xls格式表格【追加】写入数据成功！
当前插入第11条数据
xls格式表格【追加】写入数据成功！
当前插入第12条数据
xls格式表格【追加】写入数据成功！
当前插入第13条数据
xls格式表格【追加】写入数据成功！
当前插入第14条数据
xls格式表格【追加】写入数据成功！
当前插入第15条数据
xls格式表格【追加】写入数据成功！
当前插入第16条数据
xls格式表格【追加】写入数据成功！
当前插入第17条数据
xls格式表格【追加】写入数据成功！
当前插入第18条数据
xls格式表格【追加】写入数据成功！
当前插入第19条数据
xls格式表格【追加】写入数据成功！
当前插入第20条数据
xls格式表格【追加】写入数据成功！
判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
雅顿 Message: Unable to locate e

当前插入第9条数据
xls格式表格【追加】写入数据成功！
当前插入第10条数据
xls格式表格【追加】写入数据成功！
当前插入第11条数据
xls格式表格【追加】写入数据成功！
当前插入第12条数据
xls格式表格【追加】写入数据成功！
当前插入第13条数据
xls格式表格【追加】写入数据成功！
当前插入第14条数据
xls格式表格【追加】写入数据成功！
当前插入第15条数据
xls格式表格【追加】写入数据成功！
当前插入第16条数据
xls格式表格【追加】写入数据成功！
当前插入第17条数据
xls格式表格【追加】写入数据成功！
当前插入第18条数据
xls格式表格【追加】写入数据成功！
当前插入第19条数据
xls格式表格【追加】写入数据成功！
当前插入第20条数据
xls格式表格【追加】写入数据成功！
判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
当前包含微博最大数量：20,n当前的值为：0, n值到5说明已无法解析出新的微博
当前微博数以达到10条
当前插入第1条数据
目前是第一行
xls格式表格【追加】写入数据成功！
当前插入第2条数据
xls格式表格【追加】写入数据成功！
当前插入第3条数据
xls格式表格【追加】写入数据成功！
当前插入第4条数据
xls格式表格【追加】写入数据成功！
当前插入第5条数据
xls格式表格【追加】写入数据成功！
当前插入第6条数据
xls格式表格【追加】写入数据成功！
当前插入第7条数据
xls格式表格【追加】写入数据成功！
当前插入第8条数据
xls格式表格【追加】写入数据成功！
当前插入第9条数据
xls格式表格【追加】写入数据成功！
当前插入第10条数据
xls格式表格【追加】写入数据成功！
当前插入第11条数据
xls格式表格【追加】写入数据成功！
当前插入第12条数据
xls格式表格【追加】写入数据成功！
当前插入第13条数据
xls格式表格【追加】写入数据成功！
当前插入第14条数据
xls格式表格【追加】写入数据成功！
当前插入第15条数据
xls格式表格【追加】写入数据成功！
当前插入第16条数据
xls格式表格【追加】写入数据成功！
当前插入第17条数据
xls格式表格【追加】写入数据成功！
当前插入第18条数据
xls格式表格【追加】写入数据成功！
当前插入第19条数据
xls格式表

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
CHIOTURE 稚优泉 Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
当前包含微博最大数量：20,n当前的值为：0, n值到5说明已无法解析出新的微博
当前微博数以达到10条
当前插入第1条数据
目前是第一行
xls格式表格【追加】写入数据成功！
当前插入第2条数据
xls格式表格【追加】写入数据成功！
当前插入第3条数据
xls格式表格【追加】写入数据成功！
当前插入第4条数据
xls格式表格【追加】写入数据成功！
当前插入第5条数据
xls格式表格【追加】写入数据成功！
当前插入第6条数据
xls格式表格【追加】写入数据成功！
当前插入第7条数据
xls格式表格【追加】写入数据成功！
当前插入第8条数据
xls格式表格【追加】写入数据成功！
当前插入第9条数据
xls格式表格【追加】写入数据成功！
当前插入第10条数据
xls格式表格【追加】写入数据成功！
当前插入第11条数据
xls格式表格【追加】写入数据成功！
当前插入第12条数据
xls格式表格【追加】写入数据成功！
当前插入第13条数据
xls格式表格【追加】写入数据成功！
当前插入第14条数据
xls格式表格【追加】写入数据成功！
当前插入第15条数据
xls格式表格【追加】写入数据成功！
当前插入第16条数据
xls格式表格【追加】写入数据成功！
当前插入第17条数据
xls格式表格【追加】写入数据成功！
当前插入第18条数据
xls格式表格【追加】写入数据成功！
当前插入第19条数据
xls格式表格【追加】写入数据成功！
当前插入第20条数据
xls格式表格【追加】写入数据成功！
判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
碧柔 Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链

...

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
Geomar Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
office Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
PAYOT Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
ARISTON Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
love beauty and planet Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
JANESEAL SEYMOUR Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
TOCCA Message: Unable to locate element: //*[@id='app']/div[1]/di

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
当前包含微博最大数量：10,n当前的值为：0, n值到5说明已无法解析出新的微博
当前包含微博最大数量：20,n当前的值为：0, n值到5说明已无法解析出新的微博
当前微博数以达到10条
当前插入第1条数据
目前是第一行
xls格式表格【追加】写入数据成功！
当前插入第2条数据
xls格式表格【追加】写入数据成功！
当前插入第3条数据
xls格式表格【追加】写入数据成功！
当前插入第4条数据
xls格式表格【追加】写入数据成功！
当前插入第5条数据
xls格式表格【追加】写入数据成功！
当前插入第6条数据
xls格式表格【追加】写入数据成功！
当前插入第7条数据
xls格式表格【追加】写入数据成功！
当前插入第8条数据
xls格式表格【追加】写入数据成功！
当前插入第9条数据
xls格式表格【追加】写入数据成功！
当前插入第10条数据
xls格式表格【追加】写入数据成功！
当前插入第11条数据
xls格式表格【追加】写入数据成功！
当前插入第12条数据
xls格式表格【追加】写入数据成功！
当前插入第13条数据
xls格式表格【追加】写入数据成功！
当前插入第14条数据
xls格式表格【追加】写入数据成功！
当前插入第15条数据
xls格式表格【追加】写入数据成功！
当前插入第16条数据
xls格式表格【追加】写入数据成功！
当前插入第17条数据
xls格式表格【追加】写入数据成功！
当前插入第18条数据
xls格式表格【追加】写入数据成功！
当前插入第19条数据
xls格式表格【追加】写入数据成功！
当前插入第20条数据
xls格式表格【追加】写入数据成功！
判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
Listerine Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
Colgate Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
BEME Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
RE:CIPE Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
REJOICE Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
MAVALA Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
LUCK FISH Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
attitude·blue Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
MEKOO Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
kaixi ber Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
the Balm Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
AN BEATS Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
color fans Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
JAN AN NO Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
MENOW Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
Laura Mercier Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
longliqi Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/div/div/a/div[2]/h4[1]

判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
当前包含微博最大数量：10,n当前的值为：0, n值到5说明已无法解析出新的微博
当前包含微博最大数量：20,n当前的值为：0, n值到5说明已无法解析出新的微博
当前微博数以达到10条
当前插入第41条数据
xls格式表格【追加】写入数据成功！
当前插入第42条数据
xls格式表格【追加】写入数据成功！
当前插入第43条数据
xls格式表格【追加】写入数据成功！
当前插入第44条数据
xls格式表格【追加】写入数据成功！
当前插入第45条数据
xls格式表格【追加】写入数据成功！
当前插入第46条数据
xls格式表格【追加】写入数据成功！
当前插入第47条数据
xls格式表格【追加】写入数据成功！
当前插入第48条数据
xls格式表格【追加】写入数据成功！
当前插入第49条数据
xls格式表格【追加】写入数据成功！
当前插入第50条数据
xls格式表格【追加】写入数据成功！
当前插入第51条数据
xls格式表格【追加】写入数据成功！
当前插入第52条数据
xls格式表格【追加】写入数据成功！
当前插入第53条数据
xls格式表格【追加】写入数据成功！
当前插入第54条数据
xls格式表格【追加】写入数据成功！
当前插入第55条数据
xls格式表格【追加】写入数据成功！
当前插入第56条数据
xls格式表格【追加】写入数据成功！
当前插入第57条数据
xls格式表格【追加】写入数据成功！
当前插入第58条数据
xls格式表格【追加】写入数据成功！
当前插入第59条数据
xls格式表格【追加】写入数据成功！
当前插入第60条数据
xls格式表格【追加】写入数据成功！
判断页面1成功 0失败  结果是=1
超话链接获取完毕，休眠2秒
ZARA HOME Message: Unable to locate element: //*[@id='app']/div[1]/div[1]/div[1]/div[4]/div/

In [43]:
check_lst = []
for idx in range(708):
    EN,CN = df['brandEN'][idx],df['brandCN'][idx]
    if type(EN) == str:
        EN = EN.strip()
    if type(CN) == str:
        CN = CN.strip()
    if pd.notnull(EN):
        output = pd.read_excel('output/'+EN+'.xls',header=0)
        if len(output) == 0:
            e = 0
        else:
            e = 1
    if pd.notnull(CN):
        output = pd.read_excel('output/'+CN+'.xls',header=0)
        if len(output) == 0:
            c = 0
        else:
            c = 1
    if e+c == 0:
        check_lst += [0]
    else:
        check_lst += [1]

In [44]:
df['hot_topic']=check_lst

In [52]:
df = df.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1)

In [55]:
df.to_csv('hot_topic.csv',encoding = 'utf_8_sig')